In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
encoded_ds = pd.read_csv("D:\ANRD FILES\MCA Study\SEM 4\Final Project\Personality prediction\PPML-MARK-4.1\mbti_cleaned.csv")
encoded_ds.head()

<>:1: SyntaxWarning: invalid escape sequence '\A'
<>:1: SyntaxWarning: invalid escape sequence '\A'
C:\Users\anuru\AppData\Local\Temp\ipykernel_19132\930838385.py:1: SyntaxWarning: invalid escape sequence '\A'
  encoded_ds = pd.read_csv("D:\ANRD FILES\MCA Study\SEM 4\Final Project\Personality prediction\PPML-MARK-4.1\mbti_cleaned.csv")


,Unnamed: 0,is_Introvert,is_Sensing,is_Thinking,is_Perceiving,type,clean posts
0,0,1,0,0,0,INFJ,moment sportscent not top ten play prankswhat ...
1,1,0,0,1,1,ENTP,find lack post alarm sex bore posit often exam...
2,2,1,0,1,1,INTP,good one cours say know bless curs absolut pos...
3,3,1,0,1,0,INTJ,dear enjoy convers day esoter gab natur univer...
4,4,0,0,1,0,ENTJ,fire anoth silli misconcept approach logic go ...


In [3]:
encoded_ds = encoded_ds[encoded_ds["clean posts"].notna()] # Consider only those posts that are not NaN
encoded_ds.head()

,Unnamed: 0,is_Introvert,is_Sensing,is_Thinking,is_Perceiving,type,clean posts
0,0,1,0,0,0,INFJ,moment sportscent not top ten play prankswhat ...
1,1,0,0,1,1,ENTP,find lack post alarm sex bore posit often exam...
2,2,1,0,1,1,INTP,good one cours say know bless curs absolut pos...
3,3,1,0,1,0,INTJ,dear enjoy convers day esoter gab natur univer...
4,4,0,0,1,0,ENTJ,fire anoth silli misconcept approach logic go ...


In [4]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 1500)
cv.fit(encoded_ds["clean posts"])

CountVectorizer(max_features=1500)

In [5]:
X = cv.transform(encoded_ds["clean posts"]).toarray()
y = encoded_ds.iloc[:, 1:5].values

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [19]:
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.base import BaseEstimator
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

class XGBoostClassifier(BaseEstimator):
    def __init__(self, n_estimators=100, learning_rate=0.1, max_depth=3, min_samples_split=2,
                 min_impurity=1e-7, subsample=1.0, colsample_bytree=1.0, reg_lambda=1.0, reg_alpha=0.0,
                 tol=1e-4, n_jobs=None, random_state=None):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_impurity = min_impurity
        self.subsample = subsample
        self.colsample_bytree = colsample_bytree
        self.reg_lambda = reg_lambda
        self.reg_alpha = reg_alpha
        self.tol = tol
        self.n_jobs = n_jobs
        self.random_state = random_state
        self.estimators_ = []
        
    def fit(self, X, y):
        rng = np.random.default_rng(seed=self.random_state)
        n_samples, n_features = X.shape
        self.estimators_ = []
        
        F_prev_train = np.zeros(n_samples)
        
        for i in range(self.n_estimators):
            residuals = y - self._sigmoid(F_prev_train)
            tree = DecisionTreeRegressor(max_depth=self.max_depth,
                                          min_samples_split=self.min_samples_split,
                                          min_impurity_decrease=self.min_impurity,
                                          random_state=rng.integers(0, np.iinfo(np.int32).max))
            sample_weights = self.subsample * rng.choice(n_samples, n_samples, replace=True)
            sample_weights = sample_weights.astype(int)  # Convert to integer type
            tree.fit(X[sample_weights], residuals[sample_weights])
            self.estimators_.append(tree)
            
            F_train = F_prev_train + self.learning_rate * tree.predict(X)
            F_prev_train = F_train
            
            if i > 0 and np.abs(self._calculate_loss(y, F_train) - self._calculate_loss(y, F_prev_train)) < self.tol:
                break
                
    def predict_proba(self, X):
        F_ensemble = np.zeros(X.shape[0])
        
        for estimator in self.estimators_:
            F_ensemble += self.learning_rate * estimator.predict(X)
        
        proba = self._sigmoid(F_ensemble)
        return np.column_stack((1 - proba, proba))
    
    def predict(self, X):
        proba = self.predict_proba(X)
        return np.argmax(proba, axis=1)
    
    def _sigmoid(self, x):
        return 1 / (1 + np.exp(-x))
    
    def _calculate_loss(self, y_true, F_pred):
        proba = self._sigmoid(F_pred)
        return -np.mean(y_true * np.log(proba) + (1 - y_true) * np.log(1 - proba))
    
    def evaluate(self, X, y):
        y_pred = self.predict(X)
        accuracy = accuracy_score(y, y_pred)
        precision = precision_score(y, y_pred)
        recall = recall_score(y, y_pred)
        f1 = f1_score(y, y_pred)
        
        return accuracy, precision, recall, f1


In [20]:
introv_model=XGBoostClassifier()
introv_model.fit(X_train, y_train[:, 0])

In [22]:
y_pred=introv_model.predict(X_test)

In [23]:
# Evaluate the model on the test data
accuracy, precision, recall, f1 = introv_model.evaluate(X_test, y_test[:, 0])

# Print evaluation metrics
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

Accuracy: 0.7804034582132565
Precision: 0.7869352869352869
Recall: 0.975775927327782
F1-score: 0.871240283879689
